In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
###### helper functions. Use them when needed #######
def get_title_from_index(index):
	return data[data.index == index]["title"].values[0]

def get_index_from_title(title):
	return data[data.title == title]["index"].values[0]
##################################################

#### STEP 1: READ CSV FILE

In [3]:
url="https://raw.githubusercontent.com/codeheroku/Introduction-to-Machine-Learning/master/Building%20a%20Movie%20Recommendation%20Engine/movie_dataset.csv"
data=pd.read_csv(url)

In [4]:
data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [5]:
data.columns

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

#### STEP 2: FEATURE SELECTION

In [6]:
features=['keywords','cast','genres','director']

#### STEP 3: DATA CLEANING

In [7]:
data.isnull().sum()

index                      0
budget                     0
genres                    28
homepage                3091
id                         0
keywords                 412
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
dtype: int64

Since the features we have selected i.e keywords,genre, director have null values, we will fill them using fillna function 

In [8]:
for feature in features:
    data[feature]=data[feature].fillna('')
    

#### STEP 4: CREATE A DATA FRAME WITH THE SELECTED FEATURES

In [9]:
def combine_features(row):
    return row['keywords']+" "+row['cast']+ " "+ row['genres']+" "+ row['director']
data['combined_features']=data.apply(combine_features,axis=1)
data['combined_features'].head()

0    culture clash future space war space colony so...
1    ocean drug abuse exotic island east india trad...
2    spy based on novel secret agent sequel mi6 Dan...
3    dc comics crime fighter terrorist secret ident...
4    based on novel mars medallion space travel pri...
Name: combined_features, dtype: object

#### STEP 5: CREATE A COUNT MATRIX FOR THIS NEW COMBINED COLUMN

In [10]:
cv= CountVectorizer()
count_matrix=cv.fit_transform(data['combined_features'])

In [11]:
print(count_matrix)

  (0, 3115)	1
  (0, 2616)	1
  (0, 4886)	1
  (0, 12386)	2
  (0, 14235)	1
  (0, 2755)	1
  (0, 12299)	1
  (0, 11517)	1
  (0, 14561)	1
  (0, 14820)	1
  (0, 11490)	1
  (0, 12134)	1
  (0, 14291)	1
  (0, 12567)	1
  (0, 7496)	1
  (0, 8831)	1
  (0, 11217)	1
  (0, 86)	1
  (0, 144)	1
  (0, 4435)	1
  (0, 11745)	1
  (0, 4566)	1
  (0, 6542)	1
  (0, 2061)	1
  (1, 86)	1
  :	:
  (4801, 10069)	1
  (4801, 5844)	1
  (4801, 252)	1
  (4801, 4098)	1
  (4801, 14796)	1
  (4801, 11361)	1
  (4801, 2978)	1
  (4801, 12036)	1
  (4801, 6138)	1
  (4802, 9659)	1
  (4802, 3812)	1
  (4802, 1788)	2
  (4802, 4210)	1
  (4802, 5181)	1
  (4802, 2912)	1
  (4802, 3821)	1
  (4802, 1069)	1
  (4802, 11185)	1
  (4802, 3681)	1
  (4802, 5399)	1
  (4802, 3894)	1
  (4802, 2056)	1
  (4802, 3093)	1
  (4802, 4502)	1
  (4802, 5900)	2


#### STEP6: COMPUTE COSINE SIMILARITY BASED ON COUNT MATRIX

In [12]:
cosine_sim=cosine_similarity(count_matrix)

In [13]:
print(cosine_sim)

[[1.         0.10540926 0.12038585 ... 0.         0.         0.        ]
 [0.10540926 1.         0.0761387  ... 0.03651484 0.         0.        ]
 [0.12038585 0.0761387  1.         ... 0.         0.11145564 0.        ]
 ...
 [0.         0.03651484 0.         ... 1.         0.         0.04264014]
 [0.         0.         0.11145564 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.04264014 0.         1.        ]]


In [14]:
movie_user_likes="Avatar"

#### STEP 7: GET INDEX OF THE MOVIE USER LIKES

In [15]:
movie_index=get_index_from_title(movie_user_likes)

In [16]:
similar_movies=list(enumerate(cosine_sim[movie_index]))

The above code will give us a list of tuples and now as our motive is to get similar movies in descending order based on similarity we will sort this list using the following code

In [17]:
sorted_similar_movies=sorted(similar_movies,key=lambda x:x[1],reverse=True)

#### STEP 8: PRINT THE FIRST 50 SIMILAR MOVIES

In [18]:
i=0
for movie in sorted_similar_movies:
    print (get_title_from_index(movie[0]))
    i=i+1
    if i>50:
        break

Avatar
Guardians of the Galaxy
Aliens
Star Wars: Clone Wars: Volume 1
Star Trek Into Darkness
Star Trek Beyond
Alien
Lockout
Jason X
The Helix... Loaded
Moonraker
Planet of the Apes
Galaxy Quest
Gravity
Alien³
Jupiter Ascending
The Wolverine
Silent Running
Zathura: A Space Adventure
Trekkies
Cargo
Wing Commander
Star Trek
Lost in Space
Babylon A.D.
The Fifth Element
Oblivion
Titan A.E.
AVP: Alien vs. Predator
The Empire Strikes Back
Dragonball Evolution
Superman Returns
Divergent
John Carter
The Black Hole
The Ice Pirates
Memoirs of an Invisible Man
Starship Troopers
The Astronaut's Wife
Machete Kills
Soldier
The Abyss
Damnation Alley
Men in Black
Space Cowboys
Space Dogs
The Time Machine
Sheena
Captain America: Civil War
Star Trek: Insurrection
Oz: The Great and Powerful


## GUI

In [19]:
from tkinter import *
def show_data():
    txt.delete(0.0, 'end')
    movie= ent.get()
    movie_user_likes= movie
    movie_index= get_index_from_title(movie_user_likes)
    #print("Movie index of given movie:", +movie_index)
    i=int(movie_index)
    Similar_movies=list(enumerate(cosine_sim[i]))
    sorted_similar_movies=sorted(Similar_movies,key= lambda x:x[1],reverse=True)
    
    i=0;
    j=0;
    List=[None]*10
    for element in sorted_similar_movies:
        #print(get_title_from_index(element[0]))
        
        s=get_title_from_index(element[0])
        List[j]=s
        j=j+1;
        
        i=i+1;
        if i>=10:
           break
    for x in range(len(List) -1,-1,-1):
        t="\n"
        txt.insert(0.0, List[x])
        txt.insert(0.0,t)

root=Tk()
root.geometry("420x300")
l1=Label(root,text="Enter Movie Name: ")
l2=Label(root,text="Top 10 Suggestions for you: ")

ent=Entry(root)
l1.grid(row=0)
l2.grid(row=2)

ent.grid(row=0,column=1)

txt=Text(root,width=50,height=13,wrap=WORD)
txt.grid(row=3, columnspan=2, sticky=W)
btn=Button(root,text="Search",bg="purple",fg="white", command=show_data)
btn.grid(row=1,columnspan=2)
root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\hp\anaconda3\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "<ipython-input-19-4862315d26ab>", line 6, in show_data
    movie_index= get_index_from_title(movie_user_likes)
  File "<ipython-input-2-e0fcb8ed6fb3>", line 6, in get_index_from_title
    return data[data.title == title]["index"].values[0]
IndexError: index 0 is out of bounds for axis 0 with size 0
